In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNet(prototxtPath, weightsPath)


In [3]:
model = load_model("mask_detector.model") #Loading our mobilenetv2 model which we have trained before

In [4]:
image = cv2.imread(r'assets\images\example_01.png') #reads image and convert them to array
(h,w) = image.shape[:2] #we just need height and width of image not extra param i.e 3(RGB color)
blob = cv2.dnn.blobFromImage(image, 1.0, (300,300), (104.0,177.0,123.0)) #means creating a compressed binary form of image, here we have passed image and
# scale factor as 1.0 means we dont have to scale it
# 2nd param is to resize image
# 3rd param is subtract this colors RGB from image so that our model can unsdertand those images (Mean Subtraction)

#### Mean Subtraction
<img src="assets/images/mean subtraction.jpg" width="500" height="500" align="left"/>

In [7]:
net.setInput(blob) #use this blob image to detect face with our dnn model
detections = net.forward() #returns numpy array of detected darker and brighter parts of image i.e Face 

In [15]:
detections

array([[[[0.        , 1.        , 0.9984427 , ..., 0.12488028,
          0.6709176 , 0.3542412 ],
         [0.        , 1.        , 0.12920395, ..., 3.9990287 ,
          4.8382664 , 4.984081  ],
         [0.        , 1.        , 0.12446587, ..., 3.994579  ,
          0.85450625, 4.978922  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [19]:
#loop over our face
for i in range(0,detections.shape[2]): #means loop it from 0 to features of face
    confidence = detections[0,0,i,2] # filter out weak detections by ensuring the confidence is greater than the minimum confidence
    if confidence > 0.5: #if chances are greater than 50% (It is a face) then perform following
        box = detections[0,0,i,3:7]*np.array([w,h,w,h]) 
        #we are taking X and Y coordinates of frame (which is framing the face) to draw a box over face, here np array is to return array with cordinates as widht and height
        (startX, startY, endX, endY) = box.astype('int') #converting each coordinate to integer
        
        #Ensure that bounding box falls withing dimension of frame
        (startX, startY) = (max(0,startX), max(0,startY)) #now our frame is the face we have to make sure that our box should not exceed the frame
        (endX, endY) = (min(w-1,endX), min(h-1,endY)) #and it should not be too small that part of face should gets out of box, 
        
        #Extract face Region of Interest, convert it from BGR to RGB, resize it to 224,224 and preprocess it
        face = image[startY:endY, startX:endX] #1) ROI
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB) #2) BGR to RGB
        face = cv2.resize(face,(224,224)) #3) Resize
        face = img_to_array(face) #4) TO array
        face = preprocess_input(face) #5) preprocess
        face = np.expand_dims(face, axis=0) #To add additional array dimension to face if it [x] then it eill be [[x]] cause model takes multidimensional array as input
        
        (mask, withoutMask) = model.predict(face)[0] #now using our model to predict if its wearing mask and getting 1st most value of array
        
        #Now determine class label i.e with mask or wihout mask and color to the bounding box over face
        label = "mask" if mask > withoutMask else "no mask"
        color = (0,255,0) if label=="mask" else (0,0,255)
        
        #include probability in bounding boxes
        label = "{}:{:.2f}%".format(label,max(mask, withoutMask)*100) # our values are in point format (0.5,0.7) so to make it real number we multiply  100 to it
        #It is a string format technique it means get only value from 0th index to start 2 positions from float number
        #i.e 0.983564 is now 98
        
        #display label and bounding boxes
        cv2.putText(image, label, (startX,startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2) #image on which to write, what to write, coordinates, FOnt, scaling of font by 0.45, color, and thicnkness
        cv2.rectangle(image, (startX,startY), (endX,endY), color, 2) #image on whcih to draw, start and end cordinates, color, thickness

cv2.imshow("output",image) #show image
cv2.waitKey(0) #wait until any key is pressed
cv2.destroyAllWindows() #after key press destoy all windows